## Create ACS Datasets for Onboard Trigger Algorithm Testing

This notebook generates anticoincidence subsystem datasets from MEGAlib simulations.

In [ ]:
from pathlib import Path
import numpy as np
import astropy.units as u
from cosiburstpy import read_yaml, ACSData

Read in the configuration .yaml file.

In [ ]:
config_file = 'produce_acs_datasets.yaml'
inputs = read_yaml(config_file)

input_path = Path(inputs['paths']['input'])
background_path = Path(inputs['paths']['background'])
mass_model_path = Path(inputs['paths']['mass_model'])
unbinned_data_path = Path(inputs['paths']['unbinned_output'])
binned_data_path = Path(inputs['paths']['binned_output'])
combined_data_path = Path(inputs['paths']['combined_output'])

time_bin_size = inputs['binning']['time_bin_size']
time_range = inputs['binning']['time_range']
energy_bins = inputs['binning']['energy_bins'] * u.keV

time_bins = list(np.arange(time_range[0], time_range[1] + time_bin_size, time_bin_size)) * u.s

Convert MEGAlib .sim files to .hdf5 files.

In [ ]:
for event_file in input_path.iterdir():
	event_data = ACSData.convert(event_file, unbinned_data_path / f'{event_file.stem.split('.')[0]}.hdf5', mass_model=mass_model_path)

Bin data in time and energy.

In [ ]:
for event_file in unbinned_data_path.iterdir():
	
	unbinned_data = ACSData.from_file(event_file)
	binned_data = unbinned_data.bin(time_bins, energy_bins)
	binned_data.write_file(binned_data_path / event_file.name)

Combine source data with background.

In [ ]:
background_files = [file for file in background_path.iterdir()]
event_files = [file for file in binned_data_path.iterdir()]

combined_data = ACSData.combine(background_files + event_files)
combined_data.write_file(combined_data_path / 'combined_acs_data.hdf5')